In [6]:
%matplotlib widget

Import libraries

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyodbc
import matplotlib.patches as patches

Set up fonts for figures

In [8]:
font = {'family' : 'serif',
        'serif' : 'Times New Roman',
        'size'   : 8}
plt.rc('font', **font)

Setting up server

In [9]:
server = 'mrwhite.intrans.iastate.edu'
database = 'sim_Andalib'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes')

sql id numbers

In [10]:
### input parameters
number_of_intersection = 8

cycle_length = 120

simulation_time = 7200
number_of_cycle = simulation_time//cycle_length
cycle_remainder = simulation_time%cycle_length

In [11]:
filePath = "C:\\Users\\ashams\\Box\\research works\\comparison signal offset optimization\\bandwidth maximization\\"
input_parameters = pd.read_excel(filePath + "input_parameters.xlsx", sheet_name = "38th_subset")

input_parameters = input_parameters.loc[:, ["Intersection Number", "intersection_id", "sg_id", "travel time" , "d1", "tt_next_intersection", "d1_next_intersection"]].copy()

input_parameters

,Intersection Number,intersection_id,sg_id,travel time,d1,tt_next_intersection,d1_next_intersection
0,9005,1,2,0.0,190,12.9,385
1,9005,1,6,0.0,210,0.0,0
2,9006,2,2,-12.9,385,12.8,585
3,9006,2,6,12.9,405,12.9,210
4,9007,3,2,-25.7,585,12.8,765
5,9007,3,6,25.7,605,12.8,405
6,9008,4,2,-38.5,765,13.1,970
7,9008,4,6,38.5,785,12.8,605
8,9009,5,2,-51.6,970,12.8,1170
9,9009,5,6,51.6,990,13.1,785


In [21]:
sim_id = 21 
run_ids = [1,2,3,5,4,6,12, 10, 16, 8, 15, 14]

intersection_loc = [200, 395, 595, 775, 980 , 1180, 1380, 1780]
intersection_id = list(range(9005, 9013))
tt_ = [13,26,39, 52, 64, 77, 103]

titles = ["Zero-offset", "AOG" , "Modified-AOG", "AOG-Upstream origin", "Delay" , "Number of Stops" , "Delay + Stops", "PrOG", "Kell Method", "BW" , "Maxband", "Multiband" ]
iteration = 1

In [13]:
def get_green_times(run_id):
        green_time = np.zeros((number_of_intersection,2, (number_of_cycle+1)*cycle_length))
        
        signal_sql_test = "SELECT [SimTime], [SC], [SG], [state] FROM [sim_Andalib].[dbo].[signalchanges] where [StudyID]="+str(sim_id)+"and [RunID]="+str(run_id)+"and [Iteration]="+str(iteration)+ "Order By [SimTime]"
        signal = pd.read_sql(signal_sql_test, cnxn)

        if(len(signal)==0): print("ERROR")

        # 1- green start, 2- yellow start, 0- Red start
        signal = signal.loc[signal.state.isin([0, 1])]
        signal = signal.loc[signal.SC.isin(intersection_id)]
        signal = signal.loc[signal.SG.isin([2,6])]
        
        signal["eog"] = (signal.SimTime - signal.SimTime*signal.state)
        signal["bog"] = (signal.SimTime - signal.SimTime*(1-signal.state))

        signal["eog"] = signal["eog"].astype(np.int64)
        signal["bog"] = signal["bog"].astype(np.int64)

        for link_ in range(len(input_parameters)):
                intersection_num = input_parameters["Intersection Number"][link_]
                intersection_ = input_parameters["intersection_id"][link_]

                sg_ = input_parameters.sg_id[link_]

                signal_ = signal.loc[(signal.SC == intersection_num) & (signal.SG == sg_)]

                if(sg_==2): bound = 0
                else: bound  = 1

                for step in range(0, len(signal_), 2):
                        bog = signal_.iloc[step, 5]

                        if(step<len(signal_) - 1):
                                eog = signal_.iloc[step+1, 4]
                        else: eog = simulation_time 

                        green_time[intersection_ -1,bound, bog:eog] = 1
        return green_time[ : , : , : simulation_time ], signal

In [14]:
green_dist = np.zeros((len(run_ids), number_of_intersection,2, simulation_time))

signal_df_list = []
for i in range(len(run_ids)): 
        green_dist[i, :, :, :], sig_df  = get_green_times(run_ids[i])
        sig_df = pd.merge(sig_df, input_parameters, how = "inner", left_on = ["SC", "SG"], right_on = ["Intersection Number", "sg_id"])

        signal_df_list.append(sig_df)

In [15]:
def plot_offset_shift(green_matrix, offset_values, axes_, bound):
        offset_values = np.array(offset_values, dtype = np.int16)  
        link_ = bound

        for i in range(green_matrix.shape[0]):
                location = input_parameters["d1"][link_]
                placeHolder_greenMatrix = np.zeros(simulation_time)
        
                for k in range(simulation_time): 
                        placeHolder_greenMatrix[(k+offset_values[i])%simulation_time] = green_matrix[i][bound][k]
        
                green_ = np.array(placeHolder_greenMatrix)

                xs = np.multiply(np.arange(1, 7201), (1 - green_))
                ys= np.ones(7200)*location
                
                axes_.scatter(xs, ys, s = 0.8, color = 'r')

                link_ = link_ + 2

In [16]:
def plot_green_lines(sig_df, ax_i, ax_j, key):
        col = 'green'
        linestyle = 'solid'
        
        if(key == "eog"): col = 'red'
                
        for i in range(len(sig_df)):
                if(sig_df.loc[i, 'd1_next_intersection']==0): continue
                
                line_style = 'solid'
                if(sig_df.loc[i, 'SG']==6): line_style = '--'
                
                xs = [sig_df.loc[i, key], sig_df.loc[i, key] + sig_df.loc[i, 'tt_next_intersection']]
                ys =  [sig_df.loc[i, 'd1'], sig_df.loc[i, 'd1_next_intersection']]
                axes[ax_i][ax_j].plot(xs, ys, color = col, linewidth = 0.5, linestyle = line_style)


In [17]:
def plot_bands(axes_, zipped_points, first_intx_loc, second_intx_loc, travel_time_last_intx): 
        for st, end in zipped_points: 
                axes_.fill([st, end, end + travel_time_last_intx, st+travel_time_last_intx], [first_intx_loc, first_intx_loc, second_intx_loc,second_intx_loc], color = 'green', alpha = 0.35)

In [18]:
x_low = 1800
x_up = 2400
y_low = 0
y_up = 2000

interx_1st_loc = [190, 210]
interx_last_loc = [1770, 1790]

last_intersection_tt = [103, -103]

In [22]:
directory = "C:\\Users\\ashams\Box\\research works\\comparison signal offset optimization\\final figure and tables\\"

fig, axes = plt.subplots(6, 2, sharex = True, sharey = True, figsize = (6.5, 8.5))
fig.tight_layout()
# fig.subplots_adjust(wspace=0.1, hspace=0.1)

for key in range(len(run_ids)): 
        offset = np.zeros(8, np.int16)
        
        i = key//2
        j = key%2
        
        for bound in range(2):
                plot_offset_shift(green_dist[key, : , :, : ], offset, axes[i][j], bound)
        
        for line_loc  in intersection_loc:
                axes[i][j].hlines(line_loc - 2, 0, simulation_time, linestyle = '--', linewidth = 0.4, color = 'green')
                
                
        sig_df = signal_df_list[key]
        
        sig_df_bog = sig_df.loc[(sig_df["bog"]>0)].reset_index()
        sig_df_eog = sig_df.loc[(sig_df["eog"]>0)].reset_index()
        
        plot_green_lines(sig_df_bog, i, j, "bog")
        plot_green_lines(sig_df_eog, i, j, "eog")
        
        axes[i][j].set_xlim(x_low, x_up)
        axes[i][j].set_ylim(y_low, y_up)
        axes[i][j].set_title(titles[key])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
# green dist
# 0- run id
# 1- Intersection number
# 2 - approach
# 3- simulation time

green_aligned = np.zeros_like(green_dist) 
green_aligned[:,  0, : , :] = green_dist[:, 0, :, :]

bandwidth = np.zeros((number_of_intersection, 2, simulation_time))

for rid in range(len(run_ids)):
        axes_i = rid//2
        axes_j = rid%2
        for j in range(2):
                for interx_id in range(1, number_of_intersection):
                        tt_test = tt_[interx_id-1] 
                        if(j==0): green_aligned[rid,  interx_id, j, : ] = np.hstack((green_dist[rid, interx_id, j,  tt_test:simulation_time], green_dist[rid, interx_id, j, 0:tt_test]) )
                        else:        green_aligned[rid,  interx_id, j, : ] = np.hstack((green_dist[rid, interx_id, j,  -tt_test:simulation_time], green_dist[rid, interx_id, j, 0: - tt_test]) )

                bandwidth = np.prod(green_aligned[rid, :, j, :], axis = 0)
                
                shifted_band = np.hstack((0, bandwidth[ :-1]))
                band_st_end = bandwidth - shifted_band

                band_start_point = np.where(band_st_end ==  1)[0]
                band_end_point  = np.where(band_st_end == -1)[0]

                band_st_end_point = zip(band_start_point, band_end_point)

                plot_bands(axes[axes_i][axes_j], band_st_end_point, interx_1st_loc[j], interx_last_loc[j], last_intersection_tt[j])

                
fig.text(-0.02, 0.48, "Position (ft)", fontsize = 11, ha = 'center' , rotation = 90)
fig.text(0.52, -0.02, "Simulation Time (sec)",  fontsize = 11, ha = 'center' )

fig.savefig(directory + "green_band.png",  dpi = 300, bbox_inches = 'tight')